# Data Loading
Using the library pypdf extract the text from the pdf file

## Importing libraries

In [1]:
# !pip install pypdf
import pandas as pd
import numpy as np
from pypdf import PdfReader

In [20]:
num_q = 210
num_o = 4
total_num_rows = num_q + (num_q*num_o)
year = 2024

## Extracting Questions from Pdf file

In [ ]:
path: str = f"data/Cuaderno_{year}_BIOLOGÍA_0_C.pdf"
reader = PdfReader(path)
full_text = []
for n in range(2, len(reader.pages)):
    page = reader.pages[n]
    text: str = page.extract_text()
    full_text += text.split("\n")

Create a Data Frame of the text

In [3]:
exam_df = pd.DataFrame(full_text, columns=["text"])
exam_df.head()

,text
0,
1,Página: 1 de 19
2,1. La barrera hematoencefálica:
3,
4,1. Es permeable a todas las sustancias pre-


## Extracting Answers from .tsv file

In [ ]:
answers_df = pd.read_table(f"data/Cuaderno_{year}_BIOLOGÍA_0_C_Respuestas.tsv")
answers_df.head()

,V0,RC,V0.1,RC.1,V0.2,RC.2,V0.3,RC.3,V0.4,RC.4
0,1,2,43,1,85,2,127,4,169,3.0
1,2,2,44,4,86,1,128,1,170,1.0
2,3,3,45,3,87,2,129,2,171,2.0
3,4,1,46,3,88,4,130,3,172,2.0
4,5,2,47,1,89,4,131,1,173,1.0


# Data Cleaning

Removing rows containing "Página" and rows that have a space " "
Aditionaly removes the last 5 elements because they are empty space

In [5]:
exam_df = exam_df[~exam_df["text"].str.contains("Página")]
exam_df = exam_df["text"].str.strip()
exam_df = exam_df.replace("", np.nan)
exam_df = exam_df.dropna()
exam_df = exam_df.reset_index(drop=True)
exam_df.tail()

1816    2. Produce patología mediante la invasión
1817                         del tejido entérico.
1818    3. Su mecanismo de transmisión es fecal -
1819                                        oral.
1820     4. Puede dar lugar a brotes y epidemias.
Name: text, dtype: object

Funcition to join the lines 

In [ ]:
def process_multi_line_str(df):
    i = 0
    while i < len(df) - 1:
        line = df.iloc[i]
        if i < len(df) -1  and line.endswith("-"):
            df.iloc[i] = df.iloc[i][:-1] + df.iloc[i + 1]
            df = df.drop(i + 1)
            df = df.reset_index(drop=True)
        else:
            i += 1
    n = 0
    while n < len(df) -1:
        if n + 1 < len(df):
            try:
                int(df.iloc[n + 1][0:1])
                n += 1
            except ValueError:
                df.iloc[n] = df.iloc[n] + " " + df.iloc[n + 1]
                df = df.drop(n +1)
                df = df.reset_index(drop= True)
        else:
            break
    return df

In [7]:
exam_df_concat = process_multi_line_str(exam_df)
exam_df_concat.head()

0                      1. La barrera hematoencefálica:
1    1. Es permeable a todas las sustancias present...
2                      2. Es permeable al O2 y al CO2.
3                         3. Es impermeable al etanol.
4                           4. Es impermeable al agua.
Name: text, dtype: object

Print the rows that do not end in "." or ":" to fix them

In [8]:
print(f"Number of expected incorrect rows = {len(exam_df_concat) - 1050}")
print(exam_df_concat[~exam_df_concat.str.endswith((".", ":"))])

Number of expected incorrect rows = 8
55        12. En la fase de despolarización rápida (fase
235                                               4. ADP
516    104. ¿Qué término describe mejor las siguiente...
559           2. Está situado en el cromosoma 13 y en el
612            4. Enfermedad de Charcot-Marie-Tooth tipo
703            4. La ultracentrifuga ción (100.000 x g –
789               4. La hemocromatosis no es hereditaria
810        162. La activación del complejo ciclina B-CDK
838              2. Mutaciones en el gen que codifica el
840             3. Presencia de una metiltransferasa del
872                4. Detección de anticuerpos en sangre
Name: text, dtype: object


In [9]:
print(exam_df_concat.iloc[550: 565])

550     3. Los ARN mensajeros y algunos ARN ribosómicos.
551    4. Los ARN mensajeros y algunos ARN no codific...
552                111. La inactivación del cromosoma X:
553                     1. Origina fenotipos en mosaico.
554            2. Favorece su inestabilidad cromosómica.
555    3. Incrementa las tasas de mutación de sus genes.
556                 4. Impide su recombinación meiótica.
557    112. Si un gen se localiza en el 13q14 signifi...
558    1. Está situado en el brazo largo del cromosom...
559           2. Está situado en el cromosoma 13 y en el
560                                                  14.
561    3. Está situado en el brazo largo del cromosom...
562    4. Está situado en el brazo corto del cromosom...
563    113. Cuando la expresión de un gen o de un par...
564                                        1. Endogamia.
Name: text, dtype: object


In [ ]:
rows_fix = [55, 559, 516, 612, 703, 810, 838, 840, ]

In [11]:
def fix_incorrect(num_row, df):
    num_row = sorted(num_row, reverse=True)
    for n in num_row:
        df.iloc[n] = df.iloc[n] + df.iloc[n + 1]
        df = df.drop(n + 1)
        df = df.reset_index(drop= True)
    return df

The expeted number of rows for an exam with 210 questions and 4 options per question is 210+(210*4) = {total_num_rows}

In [12]:
columnas_base = ['V0', 'RC']
columnas_apiladas = ['V0.1', 'B1', 'V0.2', 'B2', '']

original_col = ["V0", "RC"]
duplicate_cols = ["V0.1", "RC.1", "V0.2", "RC.2", "V0.3", "RC.3", "V0.4", "RC.4"]

answers_df_list = [answers_df[original_col]]

for i in range(0, len(duplicate_cols), 2):
    pair_cols = duplicate_cols[i:i+2]
    df_pair_col = answers_df[pair_cols].rename(columns={pair_cols[0]: "V0", pair_cols[1]: "RC"})
    answers_df_list.append(df_pair_col)

answers_df_clean = pd.concat(answers_df_list, ignore_index=True)

print(answers_df_clean)

      V0   RC
0      1  2.0
1      2  2.0
2      3  3.0
3      4  1.0
4      5  2.0
..   ...  ...
205  206  NaN
206  207  4.0
207  208  4.0
208  209  1.0
209  210  2.0

[210 rows x 2 columns]


In [18]:
exam_df_fixed = fix_incorrect(rows_fix, exam_df_concat)
print(len(exam_df_fixed))

1050


In [16]:
exam_df_fixed = exam_df_fixed.to_frame()
groups = pd.Series((exam_df_fixed.index // 5) +1)
exam_df_fixed["group"] = groups  
exam_df_fixed["option_num"] = exam_df_fixed.groupby("group").cumcount() + 1
exam_df_pivot = exam_df_fixed.pivot(index="group", columns="option_num", values="text")
exam_df_pivot = exam_df_pivot.reset_index()
exam_df_pivot = exam_df_pivot.rename_axis(None, axis=1).rename(columns={
    1: "Question",
    2: "Option 1",
    3: "Option 2",
    4: "Option 3",
    5: "Option 4"  # Si tienes 4 opciones, ajusta según corresponda
})
exam_df_pivot = exam_df_pivot.drop(columns=["group"])

exam_df_pivot.head()

,Question,Option 1,Option 2,Option 3,Option 4
0,1. La barrera hematoencefálica:,1. Es permeable a todas las sustancias present...,2. Es permeable al O2 y al CO2.,3. Es impermeable al etanol.,4. Es impermeable al agua.
1,2. El espacio subaracnoideo se encuentra:,1. Entre la aracnoides y la duramadre.,2. Entre la aracnoides y la piamadre.,3. Lleno de sangre.,4. En el sistema nervioso periférico.
2,3. La endolinfa:,1. Rellena el laberinto óseo del oído interno.,2. Contiene baja concentración de K+.,3. Rellena el laberinto membranoso del oído in...,4. Contiene alta concentración de Na+.
3,4. Las células olfatorias humanas:,1. Son células nerviosas bipolares.,2. Son células nerviosas multipolares.,3. Son células nerviosas unipolares.,4. Son células epiteliales.
4,5. La esclerosis múltiple es una patología que...,1. Hipermielinización de los axones con pérdid...,2. Desmielinización de los axones con pérdida ...,3. Incremento en la resistencia eléctrica axonal.,4. Reducción en la capacitancia axonal.


In [17]:
clean_df = pd.concat([exam_df_pivot, answers_df_clean], axis=1)
clean_df = clean_df.drop(columns=["V0"])
clean_df.head()

,Question,Option 1,Option 2,Option 3,Option 4,RC
0,1. La barrera hematoencefálica:,1. Es permeable a todas las sustancias present...,2. Es permeable al O2 y al CO2.,3. Es impermeable al etanol.,4. Es impermeable al agua.,2.0
1,2. El espacio subaracnoideo se encuentra:,1. Entre la aracnoides y la duramadre.,2. Entre la aracnoides y la piamadre.,3. Lleno de sangre.,4. En el sistema nervioso periférico.,2.0
2,3. La endolinfa:,1. Rellena el laberinto óseo del oído interno.,2. Contiene baja concentración de K+.,3. Rellena el laberinto membranoso del oído in...,4. Contiene alta concentración de Na+.,3.0
3,4. Las células olfatorias humanas:,1. Son células nerviosas bipolares.,2. Son células nerviosas multipolares.,3. Son células nerviosas unipolares.,4. Son células epiteliales.,1.0
4,5. La esclerosis múltiple es una patología que...,1. Hipermielinización de los axones con pérdid...,2. Desmielinización de los axones con pérdida ...,3. Incremento en la resistencia eléctrica axonal.,4. Reducción en la capacitancia axonal.,2.0
